## 환경 설정

In [ ]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# LangGraph 맛보기

## 1. 기본 예제

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from IPython.display import Image, display

# 1. 상태 정의
class MyState(TypedDict):
    name: str
    is_morning: bool

# 2. 노드 함수 정의
def greet_user(state: MyState) -> MyState:
    print(f"Hi, {state['name']}!")
    return state

def say_good_morning(state: MyState) -> MyState:
    print("Good morning!")
    return state

def say_hello(state: MyState) -> MyState:
    print("Hello!")
    return state

# 3. 조건 함수 정의
def is_morning(state: MyState) -> Literal["morning", "not_morning"]:
    return "morning" if state["is_morning"] else "not_morning"

# 4. 그래프 구성
builder = StateGraph(MyState)

builder.add_node("greet_user", greet_user)
builder.add_node("say_good_morning", say_good_morning)
builder.add_node("say_hello", say_hello)

builder.add_edge(START, "greet_user")
builder.add_conditional_edges(
    "greet_user",
    is_morning,
    {
        "morning": "say_good_morning",
        "not_morning": "say_hello",
    },
)
builder.add_edge("say_good_morning", END)
builder.add_edge("say_hello", END)

# 5. 그래프 컴파일
graph = builder.compile()

# 6. 그래프 시각화
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# 그래프 실행
graph.invoke({"name": "Bob", "is_morning": True})

In [ ]:
# 그래프 실행
for step in graph.stream({"name": "Bob", "is_morning": False}, stream_mode="values"):
    print(step)
    print("---"*10)

In [ ]:
# 그래프 실행
for step in graph.stream({"name": "Bob", "is_morning": False}, stream_mode="updates"):
    print(step)
    print("---"*10)

## 2. 고급 기능 사용 예제

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, AnyMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field

# 시스템 프롬프트 정의
SUMMARY_PROMPT = "You are a helpful assistant that summarizes the following text."
EVALUATE_PROMPT = "You are a helpful assistant that evaluates the quality of a summary.\n" \
                  "You must provide a quality score between 0.0 and 1.0, where 0.0 is the lowest quality and 1.0 is the highest quality."
IMPROVE_PROMPT = "You are a helpful assistant that enhances low-quality summaries generated by AI.\n" \
                 "Your goal is to rewrite them to be clearer, more accurate, and more natural."

# 출력 구조화
class Summary(BaseModel):
    summary: Annotated[str, Field(description="The summary of the text")]

class Evaluation(BaseModel):
    quality: Annotated[float, Field(description="The quality of the summary", ge=0, le=1)]

# 모델 정의
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.5
)

# 상태 정의
class SummaryState(TypedDict):
    text: str
    summary: str
    quality: float
    finalized: bool
    iteration: int
    messages: Annotated[list[AnyMessage], add_messages]

# 1. 요약 노드
def summarize_text(state: SummaryState) -> Command:
    messages = [
        SystemMessage(content=SUMMARY_PROMPT),
        HumanMessage(content=f"Please summarize the following text: {state['text']}\n\nSummary:")
    ]
    response = llm.with_structured_output(Summary).invoke(messages)

    print(f"[summarize_text] 요약 완료")
    return Command(
        goto="evaluate_summary",
        update={
            "summary": response.summary, 
            "iteration": 0,
            "messages": messages + [AIMessage(content=response.summary)]
        }
    )

# 2. 품질 평가 노드
def evaluate_summary(state: SummaryState) -> Command:
    messages = [
        SystemMessage(content=EVALUATE_PROMPT),
        HumanMessage(content=f"The text is: {state['text']}\n\nPlease evaluate the following summary: {state['summary']}")
    ]
    response = llm.with_structured_output(Evaluation).invoke(messages)

    print(f"[evaluate_summary] 평가 결과: {response.quality}")

    # 품질에 따라 다음 노드 분기 및 상태 업데이트
    return Command(
        goto="finalize_summary" if response.quality > 0.8 or state["iteration"] > 3 else "improve_summary",
        update={
            "quality": response.quality, 
            "messages": messages + [AIMessage(content=str(response.quality))]
        }
    )

# 3. 개선 노드
def improve_summary(state: SummaryState) -> Command:
    messages = [
        SystemMessage(content=IMPROVE_PROMPT),
        HumanMessage(content=f"The text is: {state['text']}\n\n"
                             f"Please enhance the following summary: {state['summary']}\n\nEnhanced Summary:")
    ]
    response = llm.with_structured_output(Summary).invoke(messages)

    print(f"[improve_summary] 요약 수정됨")
    return Command(
        goto="evaluate_summary",
        update={
            "summary": response.summary, 
            "iteration": state["iteration"] + 1,
            "messages": messages + [AIMessage(content=response.summary)]
        }
    )

# 4. 최종화 노드
def finalize_summary(state: SummaryState) -> Command:
    print(f"[finalize_summary] 최종 요약 완료")
    return Command(
        goto=END,
        update={"finalized": True}
    )

# 그래프 생성
builder = StateGraph(SummaryState)

builder.add_node("summarize_text", summarize_text)
builder.add_node("evaluate_summary", evaluate_summary)
builder.add_node("improve_summary", improve_summary)
builder.add_node("finalize_summary", finalize_summary)

builder.add_edge(START, "summarize_text")

graph = builder.compile()

# 테스트 케이스
text = """
The app is useful. I use it every day. Some features are hard to find. But I still use it. It’s okay.
"""
for step in graph.stream({"text": text}, stream_mode="updates"):
    print(step)
    print("---"*10)

## 3. Send 예제

> Map-Reduce 패턴 구현

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.documents import Document
from typing import TypedDict, List, Annotated
from operator import add
from IPython.display import Image, display
from pprint import pprint

# 전체 상태 정의
class OverallState(TypedDict):
    docs: List[Document]
    summaries: Annotated[List[str], add]

# 로컬 상태 정의
class DocState(TypedDict):
    doc: Document

# 라우터 함수
def map_router(state: OverallState) -> List[Send]:
    return [Send("map_node", {"doc": doc}) for doc in state["docs"]]

# 맵 노드 함수
def map_node(state: DocState) -> OverallState:
    doc = state["doc"]
    summary = f"{doc.metadata['source']}: {doc.page_content[:30]}..."
    return {"summaries": [summary]}

# 그래프 생성
builder = StateGraph(OverallState)

builder.add_node("map_node", map_node)
builder.add_conditional_edges(START, map_router, ["map_node"])
builder.add_edge("map_node", END)

graph = builder.compile()

# 테스트
docs = [
    Document(page_content="LangChain is a framework for building LLM-powered apps.", metadata={"source": "intro"}),
    Document(page_content="StateGraph enables structured workflows with shared state.", metadata={"source": "graph"}),
    Document(page_content="Send can dynamically route data to other nodes.", metadata={"source": "send"}),
]
result = graph.invoke({"docs": docs})
pprint(result)

# ReAct Agent 구현

## 1. Tool 바인딩 활용
> `ToolNode`와 `Tool` 바인딩을 사용한 Reasoning + Acting 패턴

In [ ]:
import requests
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.types import Send
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image, display


# 검색 도구
search_tool = DuckDuckGoSearchRun(name="search_tool")

# 날씨 도구
@tool
def weather_tool(latitude: float, longitude: float) -> str:
    """Get current weather information for a specific latitude/longitude.
    
    Args:
        latitude: Latitude coordinate
        longitude: Longitude coordinate
        
    Returns:
        Weather information for the coordinates
    """
    try:
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true",
            "hourly": "relative_humidity_2m"
        }
        resp = requests.get(url, params=params, timeout=5)
        resp.raise_for_status()
        data = resp.json()
        # 기온
        temp = data.get("current_weather", {}).get("temperature")
        # 습도: 가장 가까운 시간의 습도 사용
        humidity = "N/A"
        if "hourly" in data and "relative_humidity_2m" in data["hourly"]:
            # 현재 시간 index 찾기
            current_time = data["current_weather"]["time"]
            times = data["hourly"]["time"]
            humidities = data["hourly"]["relative_humidity_2m"]
            if current_time in times:
                idx = times.index(current_time)
                humidity = humidities[idx]
            else:
                humidity = humidities[0]
        return f"현재 기온: {temp}°C, 습도: {humidity}%"
    except Exception as e:
        return f"날씨 API 호출 오류: {str(e)}"

# LLM 정의
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0
)

# 도구 정의
tools = [search_tool, weather_tool]

# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

# 시스템 메시지 정의
system_message = SystemMessage(content="""You are a helpful AI assistant that can use tools to answer questions.

Available tools:
- search_tool: For searching information  
- weather_tool: For weather information

When you need to use a tool, think step by step:
1. Identify what information you need
2. Choose the appropriate tool
3. Use the tool with the correct parameters
4. Analyze the results
5. Provide a helpful response

If you can answer directly without tools, do so. Always be helpful and accurate.""")

# 상태 정의
class AgentState(MessagesState):
    ...

# 에이전트 노드
def agent_node(state: AgentState) -> List[Send]:
    messages = [system_message] + state["messages"]
    response = llm_with_tools.invoke(messages)
    
    print(f"[Agent] 응답 생성: {response.content[:20]}...")
    if response.tool_calls:
        print(f"[Agent] 도구 호출: {[tool['name'] for tool in response.tool_calls]}")

        return [Send("tools", {
                "tool_name": tool_call["name"], 
                "parameters": tool_call.get("parameters", {})
            }) for tool_call in response.tool_calls
        ]
    return {"messages": [response]}

# 그래프 구성
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)      # 에이전트 노드
workflow.add_node("tools", ToolNode(tools)) # 도구 실행 노드

# 엣지 설정
workflow.add_edge(START, "agent")
workflow.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "tools",
        "end": END
    }
)
workflow.add_edge("tools", "agent")

# 그래프 컴파일
react_agent = workflow.compile()

# 그래프 시각화
display(Image(react_agent.get_graph().draw_mermaid_png()))

In [ ]:
# === 테스트 예제들 ===

# 테스트 1: 계산 질문
def test_calculation():
    print("=== 테스트 1: 계산 질문 ===")
    question = "What is 25 * 4 + 18?"
    
    result = react_agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    print(f"\n질문: {question}")
    print(f"최종 답변: {result['messages'][-1].content}")
    print("-" * 50)

# 테스트 2: 검색 질문  
def test_search():
    print("=== 테스트 2: 검색 질문 ===")
    question = "What is Python programming language?"
    
    result = react_agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    print(f"\n질문: {question}")
    print(f"최종 답변: {result['messages'][-1].content}")
    print("-" * 50)

# 테스트 3: 날씨 질문
def test_weather():
    print("=== 테스트 3: 날씨 질문 ===") 
    question = "What's the weather like in Seoul?"
    
    result = react_agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    print(f"\n질문: {question}")
    print(f"최종 답변: {result['messages'][-1].content}")
    print("-" * 50)

# 테스트 4: 복합 질문 (여러 도구 사용)
def test_complex():
    print("=== 테스트 4: 복합 질문 ===")
    question = "If I have 100 dollars and buy 3 items that cost 15 dollars each, how much money do I have left? Also, what's the weather in Tokyo?"
    
    # stream 모드로 단계별 과정 확인
    print(f"질문: {question}\n")
    
    for chunk in react_agent.stream(
        {"messages": [HumanMessage(content=question)]},
        stream_mode="updates"
    ):
        node_name = list(chunk.keys())[0]
        if node_name == "agent":
            message = chunk[node_name]["messages"][0]
            print(f"🤖 Agent: {message.content[:100]}...")
            if hasattr(message, 'tool_calls') and message.tool_calls:
                for tool_call in message.tool_calls:
                    print(f"   🔧 도구 호출: {tool_call['name']}")
        elif node_name == "tools":
            tool_messages = chunk[node_name]["messages"]
            for msg in tool_messages:
                if hasattr(msg, 'content'):
                    print(f"   ⚙️  도구 결과: {msg.content}")
        print()

# 모든 테스트 실행
def run_all_tests():
    test_calculation()
    test_search()
    test_weather() 
    test_complex()

# 테스트 실행
run_all_tests()
